Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

In [ ]:
# This is only needed for this notebook to work
import sys
from pathlib import Path

# Add the parent directory of `src` to the path
sys.path.append(str(Path.cwd() / "src"))

# Text2SQL with AutoGen & Azure OpenAI

This notebook demonstrates how the AutoGen Agents can be integrated with Azure OpenAI to answer questions from the database based on the schemas provided. 

A multi-shot approach is used for SQL generation for more reliable results and reduced token usage. More details can be found in the README.md.

### Dependencies

To install dependencies for this demo. Navigate to the autogen directory:

`uv sync`

If you need a differnet connector to TSQL.

`uv sync --extra <DATABASE ENGINE>`

In [ ]:
import dotenv
import logging
from autogen_text_2_sql import AutoGenText2Sql, UserMessagePayload
from autogen_text_2_sql.state_store import InMemoryStateStore

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
dotenv.load_dotenv()

## Bot setup

In [ ]:
# The state store allows AutoGen to store the states in memory across invocation. Whilst not neccessary, you can replace it with your own implementation that is backed by a database or file system. 
agentic_text_2_sql = AutoGenText2Sql(state_store=InMemoryStateStore(), use_case="Analysing sales data")

### Generated Queries

In [ ]:
async for message in agentic_text_2_sql.process_user_message(thread_id="1", message_payload=UserMessagePayload(user_message="what are the total sales")):
    logging.info("Received %s Message from Text2SQL System", message)